<a href="https://colab.research.google.com/github/charviadikar/AI_lab_1BM22CS012/blob/main/1BM22CS012_Week9_FOL_Resolution_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from itertools import combinations

def unify(x, y, theta={}):
    """Unify two predicates using a substitution (theta)."""
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    else:
        return None

def unify_var(var, x, theta):
    """Unify variable."""
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def resolve(clause1, clause2):
    """Resolve two clauses and return the resolvents."""
    resolvents = []
    for predicate1 in clause1:
        for predicate2 in clause2:
            theta = unify(predicate1, negate(predicate2))
            if theta is not None:
                new_clause = (substitute(clause1, theta) | substitute(clause2, theta)) - {predicate1, predicate2}
                resolvents.append(frozenset(new_clause))
    return resolvents

def negate(predicate):
    """Negate a predicate."""
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def substitute(clause, theta):
    """Apply substitutions to a clause."""
    return {substitute_predicate(p, theta) for p in clause}

def substitute_predicate(predicate, theta):
    """Apply substitution to a single predicate."""
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])

def resolution(kb, query):
    """Perform resolution to prove the query."""
    negated_query = frozenset({negate(query)})
    clauses = kb | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

kb = {
    frozenset({('Mother', 'Leela', 'Oshin')}),
    frozenset({('Alive', 'Leela')}),
    frozenset({('not','Mother', 'x','y')}),
    frozenset({('Parent','x','y')}),
    frozenset({('not','Parent', 'w', 'z')}),
    frozenset({('not','Alive','w','z')}),
    frozenset({('Older','w','z')}),
}

# Query: Prove John likes peanuts
query = ('Older', 'Leela', 'Older')

# Run resolution
result = resolution(kb, query)
if result:
    print("Proved by resolution: Hence, Leela is older than Oshin.")
else:
    print("Cannot prove. Leela is not older than Oshin.")




Proved by resolution: Hence, Leela is older than Oshin.
